In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import earthaccess
import h5netcdf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyinterp.backends.xarray  # Module that handles the filling of undefined values.
import pyinterp.fill
import seaborn as sns
import xarray as xr
from matplotlib.patches import Rectangle

In [2]:
auth = earthaccess.login()

In [4]:
# plot = transect["syncoccus_moana"].plot(y="lat")

In [5]:
tspan = ("2024-04-01", "2025-08-31")

In [6]:
results_moana = earthaccess.search_data(
    short_name="PACE_OCI_L3M_MOANA",
    granule_name="*.Day.*0p1deg*",  # Daily: Day | Resolution: 0p1deg or 4 (for 4km)
    temporal=tspan,
)

In [7]:
def time_from_attr(ds):
    """Set the time attribute as a dataset variable
    Args:
        ds: a dataset corresponding to one or multiple Level-2 granules
    Returns:
        the dataset with a scalar "time" coordinate
    """
    datetime = ds.attrs["time_coverage_start"].replace("Z", "")
    ds["date"] = ((), np.datetime64(datetime, "ns"))
    ds = ds.set_coords("date")
    return ds

In [8]:
path_files = earthaccess.open(results_moana)

QUEUEING TASKS | :   0%|          | 0/376 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/376 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/376 [00:00<?, ?it/s]

In [9]:
dataset_moana = xr.open_mfdataset(
    path_files, preprocess=time_from_attr, combine="nested", concat_dim="date"
)
dataset_moana

<xarray.Dataset> Size: 7GB
Dimensions:          (date: 376, lat: 1400, lon: 1100, rgb: 3,
                      eightbitcolor: 256)
Coordinates:
  * lat              (lat) float32 6kB 69.95 69.85 69.75 ... -69.85 -69.95
  * lon              (lon) float32 4kB -84.95 -84.85 -84.75 ... 24.85 24.95
  * date             (date) datetime64[ns] 3kB 2024-04-01T11:01:27 ... 2025-0...
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    prococcus_moana  (date, lat, lon) float32 2GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    syncoccus_moana  (date, lat, lon) float32 2GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    picoeuk_moana    (date, lat, lon) float32 2GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    palette          (date, rgb, eightbitcolor) uint8 289kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/62)
    product_name:                      PACE_OCI.20240401.L3m.DAY.MOANA.V3_0.0...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    cdm_data_type:                     grid
    identifier_product_doi_authority:  http://dx.doi.org
    identifier_product_doi:            10.5067/PACE/OCI/L3M/MOANA/3.0
    data_bins:                         166600
    data_minimum:                      -inf
    data_maximum:                      inf

In [3]:
# lon_val = -30
# transect = dataset_phy.sel(lon=amt_lon, method="nearest")

In [10]:
dataset_moana["prococcus_moana"] = dataset_moana["prococcus_moana"].clip(
    min=dataset_moana["prococcus_moana"].attrs["valid_min"],
    max=dataset_moana["prococcus_moana"].attrs["valid_max"],
)
dataset_moana["syncoccus_moana"] = dataset_moana["syncoccus_moana"].clip(
    min=dataset_moana["syncoccus_moana"].attrs["valid_min"],
    max=dataset_moana["syncoccus_moana"].attrs["valid_max"],
)
dataset_moana["picoeuk_moana"] = dataset_moana["picoeuk_moana"].clip(
    min=dataset_moana["picoeuk_moana"].attrs["valid_min"],
    max=dataset_moana["picoeuk_moana"].attrs["valid_max"],
)

In [11]:
dataset_phy = dataset_moana.drop_vars(["palette"])

In [12]:
dataset_phy_mean = dataset_phy.resample(date="1M").mean("date")

In [13]:
dataset_phy_mean = dataset_phy_mean.astype(np.float64)
dataset_norm = (
    (dataset_phy_mean - dataset_phy_mean.min())
    / (dataset_phy_mean.max() - dataset_phy_mean.min())
)
data_norm = dataset_norm.to_dataarray()
data_norm = data_norm.sel(
    variable=["syncoccus_moana", "picoeuk_moana", "prococcus_moana"]
)

data_norm

<xarray.DataArray (variable: 3, date: 14, lat: 1400, lon: 1100)> Size: 517MB
dask.array<getitem, shape=(3, 14, 1400, 1100), dtype=float64, chunksize=(1, 1, 512, 1024), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float32 6kB 69.95 69.85 69.75 69.65 ... -69.75 -69.85 -69.95
  * lon       (lon) float32 4kB -84.95 -84.85 -84.75 ... 24.75 24.85 24.95
  * date      (date) datetime64[ns] 112B 2024-04-30 2024-05-31 ... 2025-05-31
  * variable  (variable) object 24B 'syncoccus_moana' ... 'prococcus_moana'

In [24]:
data_norm

<xarray.DataArray (variable: 3, date: 14, lat: 1400, lon: 1100)> Size: 517MB
dask.array<getitem, shape=(3, 14, 1400, 1100), dtype=float64, chunksize=(1, 1, 512, 1024), chunktype=numpy.ndarray>
Coordinates:
  * lat       (lat) float32 6kB 69.95 69.85 69.75 69.65 ... -69.75 -69.85 -69.95
  * lon       (lon) float32 4kB -84.95 -84.85 -84.75 ... 24.75 24.85 24.95
  * date      (date) datetime64[ns] 112B 2024-04-30 2024-05-31 ... 2025-05-31
  * variable  (variable) object 24B 'syncoccus_moana' ... 'prococcus_moana'

In [47]:
# Load and deduplicate lat/lon points
points_df = pd.read_csv("AMT28_underway_latlon.csv").drop_duplicates(subset=["Lat", "Lon"]).reset_index(drop=True)
points_df["point"] = np.arange(len(points_df))

# Create point-indexed lat/lon DataArrays
lat_da = xr.DataArray(points_df["Lat"].values, dims="point")
lon_da = xr.DataArray(points_df["Lon"].values, dims="point")

# Get list of variables from the data_norm dimension
varnames = list(data_norm["variable"].values)

# Storage for output DataFrames
all_dfs = []

for varname in varnames:
    print(f"Interpolating: {varname}")

    # Subset to the single variable (3D: date, lat, lon)
    da = data_norm.sel(variable=varname)

    # Vectorized interpolation (date, point)
    interp_result = da.interp(lat=lat_da, lon=lon_da).compute()

    # Convert to DataFrame and merge lat/lon back in
    interp_df = interp_result.to_dataframe(name=varname).reset_index()
    interp_df = interp_df.merge(points_df, on="point")[["date", "Lat", "Lon", varname]]

    all_dfs.append(interp_df)

# Merge all variable DataFrames on date/lat/lon
final_df = all_dfs[0]
for df in all_dfs[1:]:
    final_df = final_df.merge(df, on=["date", "Lat", "Lon"])


Interpolating: syncoccus_moana
Interpolating: picoeuk_moana
Interpolating: prococcus_moana


In [48]:
final_df

,date,Lat,Lon,syncoccus_moana,picoeuk_moana,prococcus_moana
0,2024-04-30,49.638,-5.502,0.000426,0.578116,0.005936
1,2024-04-30,49.581,-5.676,0.002015,0.549661,0.027168
2,2024-04-30,49.501,-5.908,0.004131,0.497193,0.027015
3,2024-04-30,49.429,-6.108,0.006967,0.387322,0.046633
4,2024-04-30,49.355,-6.332,0.001744,0.491480,0.015977
...,...,...,...,...,...,...
9935,2025-05-31,-47.860,-52.236,NaN,NaN,NaN
9936,2025-05-31,-47.933,-52.353,NaN,NaN,NaN
9937,2025-05-31,-48.014,-52.436,NaN,NaN,NaN
9938,2025-05-31,-48.115,-52.572,NaN,NaN,NaN
